In [1]:
import os
import torch
import numpy as np
import pandas as pd

from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F

from io import open
import glob
import os
import string

In [4]:

all_letters = string.ascii_letters[:26]
n_letters = len(all_letters)

n_categories = 26 #number of letters in english language

# Find letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Just for demonstration, turn a letter into a <1 x n_letters> Tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor

# Turn a line into a <line_length x 1 x n_letters>,
# or an array of one-hot letter vectors
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor


In [5]:
file_path = "C:/Users/cowpo/Downloads/dataset.txt"
df = pd.read_csv(file_path, header = None)

df.head()

words = df[1]

In [6]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size, hidden_size)
        self.h2h = nn.Linear(hidden_size, hidden_size) #may be vanishing or exploding
        self.h2o = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden): #this hidden parameter is h_{t-1}
        hidden = F.tanh(self.i2h(input) + self.h2h(hidden)) #updates hidden layer to use for next step
        output = self.h2o(hidden)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size) # 1 * hidden_size matrix of zeroes

n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

In [7]:
#example input
input = letterToTensor('p')
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input, hidden) #output has probabilies. This is applying the forward method


In [8]:
#decides our guess

def letterFromOutput(output):
    top_n, top_i = output.topk(1) #value and index of our guess
    letter_i = top_i[0].item() #actual index
    return all_letters[letter_i], letter_i

print(letterFromOutput(output)) #example



('o', 14)


In [20]:
import random

def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

def randomTrainingExample():
    input = '_____'
    word = randomChoice(words)
    word_tensor = lineToTensor(word)
    return input, word, word_tensor

for i in range(10):
    input, word, word_tensor = randomTrainingExample()
    print('input =', input, '/ word =', word)

input = _____ / word = comal
input = _____ / word = snobs
input = _____ / word = coarb
input = _____ / word = gonif
input = _____ / word = prexy
input = _____ / word = firer
input = _____ / word = deave
input = _____ / word = gobbo
input = _____ / word = laced
input = _____ / word = toits


In [ ]:
criterion = nn.CrossEntropyLoss()
learning_rate = 0.005
